In [1]:
from pyasn1.type import univ, namedtype, namedval, tag, char
from pyasn1.codec.ber import encoder
from pyasn1.codec.ber import decoder
from trustmessages import *
import itertools

In [2]:
sq1 = Query()
sq1["cmp"] = Comparison()
sq1["cmp"]["op"] = "lt"
sq1["cmp"]["value"] = Value()
sq1["cmp"]["value"]["date"] = 50

In [3]:
sq2 = Query()
sq2["log"] = Logical()
sq2["log"]["op"] = "and"
sq2["log"]["l"] = Query()
sq2["log"]["l"]["cmp"] = Comparison()
sq2["log"]["l"]["cmp"]["op"] = "eq"
sq2["log"]["l"]["cmp"]["value"] = Value()
sq2["log"]["l"]["cmp"]["value"]["source"] = "alice"
sq2["log"]["r"] = Query()
sq2["log"]["r"]["cmp"] = Comparison()
sq2["log"]["r"]["cmp"]["op"] = "eq"
sq2["log"]["r"]["cmp"]["value"] = Value()
sq2["log"]["r"]["cmp"]["value"]["target"] = "bob"

In [4]:
q = Query()
q["log"] = Logical()
q["log"]["l"] = Query()
q["log"]["l"]["cmp"] = Comparison()
q["log"]["l"]["cmp"]["op"] = "eq"
q["log"]["l"]["cmp"]["value"] = Value()
q["log"]["l"]["cmp"]["value"]["service"] = "seller"

q["log"]["op"] = "and"

q["log"]["r"] = Query()
q["log"]["r"]["log"] = Logical()
q["log"]["r"]["log"]["op"] = "or"
q["log"]["r"]["log"]["l"] = Query()
q["log"]["r"]["log"]["l"]["cmp"] = Comparison()
q["log"]["r"]["log"]["l"]["cmp"]["op"] = "eq"
q["log"]["r"]["log"]["l"]["cmp"]["value"] = Value()
q["log"]["r"]["log"]["l"]["cmp"]["value"]["source"] = "charlie"
q["log"]["r"]["log"]["r"] = Query()
q["log"]["r"]["log"]["r"]["cmp"] = Comparison()
q["log"]["r"]["log"]["r"]["cmp"]["op"] = "eq"
q["log"]["r"]["log"]["r"]["cmp"]["value"] = Value()
q["log"]["r"]["log"]["r"]["cmp"]["value"]["source"] = "david"

In [5]:
substrate = encoder.encode(q)
d, e = decoder.decode(substrate, asn1Spec=Query())
assert(d == q)
assert(q.prettyPrint() == d.prettyPrint())

In [6]:
qtm = QtmDb()

In [7]:
assert(all(t["date"] < 50
           for t in itertools.ifilter(trustutils.create_predicate(sq1), qtm.ASSESSMENT_DB)))

In [8]:
assert(all(t["source"] == "alice" and t["target"] == "bob"
           for t in itertools.ifilter(trustutils.create_predicate(sq2), qtm.ASSESSMENT_DB)))

In [9]:
assert(all(t["service"] == "seller" and (t["source"] == "charlie" or t["source"] == "david")
           for t in itertools.ifilter(trustutils.create_predicate(q), qtm.ASSESSMENT_DB)))